# 1. Introdução

A **Transformada de Fourier** é uma relação matemática que permite passar uma função no domínio do tempo (ou do espaço) para o domínio da frequência (ou do número de onda), e vice versa. Primeiramente, vamos recapitular a equação da Série de Fourier complexa. Seja f periódica de período T e seccionalmente contínua. Sua série de Fourier complexa é dada por:

$\large \displaystyle{f(t) = \sum_{n=-\infty}^{\infty} e^{\frac{i2\pi n t}{T}}\left (\frac{1}{T} \int\limits_{-T/2}^{T/2}f(\tau)e^{\frac{-i2\pi n\tau}{T}} d\tau \right)}$

Vamos substituir $\frac{2\pi}{T}$ por $\Delta\omega$ e $n\Delta\omega$ por $\omega_{n}$, de forma a deixar a expressão anterior parecida com uma soma de Riemann:

$\large \displaystyle{f(t) = \sum\limits_{n=-\infty}^{\infty}e^{i\omega_{n}t}\frac{\Delta\omega}{2\pi}\int\limits_{-\pi/\Delta\omega}^{\pi/\Delta\omega}f(\tau)e^{-i\omega_{n}\tau} d\tau}$

**Integral de Riemann:** Lembre-se que: $\large \displaystyle{\int\limits_{a}^{b} g(x) dx = lim_{N\rightarrow\infty,\Delta x \rightarrow 0} \sum\limits_{n=0}^{N}g(n\Delta x + a) \Delta x}$

Se imaginarmos que uma função aperiódica pode ser interpretada como uma função periódica cujo período T é infinito, observamos que no limite $T \rightarrow \infty$, temos $\Delta\omega \rightarrow 0$, o que nos dá:

$\large \displaystyle{f(t) = \frac{1}{2\pi}\int\limits_{-\infty}^{\infty}e^{i\omega t} \hat{f}(\omega)d\omega}$

A equação acima define a Transformada Inversa de Fourier. A expressão para $\hat{f}(\omega)$ determina a Transformada de Fourier:

$\large \displaystyle{\hat{f}(\omega) = \int\limits_{-\infty}^{\infty}f(t)e^{-i\omega t} dt}$

**Exemplo.** Considere a função $\displaystyle{f(t) = \begin{cases}A, & -a \le t \le a \\ 0, & caso & contrário \end{cases}}$.

A Transformada de Fourier é dada por:

$\large \displaystyle{\hat{f}(\omega) = \int\limits_{-\infty}^{\infty}f(t)e^{-i\omega t} dt = A \int\limits_{-a}^{a}e^{-i\omega t} = A\frac{e^{-i\omega t}}{-i\omega t}|_{-a}^{a} = \frac{2A}{\omega}sen(\omega a) = 2Aa\frac{sen(\omega a)}{\omega a}}$

Vamos agora passar para exemplos de implementação em Python.

In [ ]:
# importando bibliotecas

import numpy as np
import matplotlib.pyplot as plt
# pip install sympy
import sympy as smp
from scipy.fft import fftfreq, fft

Antes de começarmos, vamos pegar a definição anterior da Transformada de Fourier e sua inversa e vamos reescrevê-la, modificando ligeiramente a nomenclatura e lembrando que $\omega = \frac{2\pi}{T} = 2\pi f$. Vamos lidar somente com transformações entre tempo e frequência, mas lembre-se que transformações com espaço e número de onda são análogas.

$\large \displaystyle{\hat{x}(f) = \int\limits_{-\infty}^{\infty}x(t)e^{-2\pi ift} dt}$

e a transformada inversa

$\large \displaystyle{x(t) = \frac{1}{2\pi}\int\limits_{-\infty}^{\infty}e^{2\pi ift}\hat{x}(f) df} $

# 2. Transformada de Fourier (tempo e frequência contínuos)

Esse primeiro caso acontece quando a forma funcional da sua série temporal é conhecida analiticamente, ou seja, você possui uma fórmula do tipo x(t) = ..., e vai de $-\infty$ a $\infty$.

Para resolver **analiticamente** (se possível) vamos utilizar a biblioteca chamada [SymPy](https://www.sympy.org/pt/index.html), que é uma biblioteca para lidar com matemática simbólica.

**Exemplo.** Encontre a transformada de Fourier de $e^{-t^{2}}$.

In [ ]:
# definindo variáveis simbólicas
t, f = smp.symbols("t f", real=True)
x = smp.exp(-t**2)
x

In [ ]:
# vamos utilizar o método integrate e escrever nossa equação
# da transformada de Fourier

xf = smp.integrate(x*smp.exp(-2*smp.pi*smp.I*f*t), (t, -smp.oo, smp.oo))
xf

In [ ]:
# podemos simplificar o resultado

xf = xf.simplify()
xf

In [ ]:
# para não precisar escrever a integral todas as vezes, podemos importar um método

from sympy.integrals.transforms import fourier_transform

fourier_transform(x, t, f).simplify()

In [ ]:
# para fins de teste, vamos recuperar a função x(t)

xf = smp.sqrt(smp.pi)*smp.exp(-smp.pi**2*f**2)
xf

In [ ]:
from sympy.integrals.transforms import inverse_fourier_transform

In [ ]:
inverse_fourier_transform(xf, f, t)

O SymPy nos poupou um bom tempo resolvendo no papel! 

**Exercício.** Agora, encontre a transformada de Fourier de $\large x(t) = cte^{-ct^2}$, onde c é uma constante positiva (dica: certifique-se que c é positiva, ao definí-la simbolicamente).

**Exercício.** Encontre a transformada de Fourier de $\large x(t) = b cos(t^2)$, onde b é uma constante positiva.

**Exercício.** Plote o gráfico do resultado do item anterior (dica: crie um vetor de frequências e fixe o valor de b).

**Exercício.** Encontre a transformada de Fourier de $\large x(t) = e^{-ct^{2}}sin{(ct)}t^{4}$, onde c é uma constante positiva.

Pare a execução da célula do exercício anterior, se você escreveu corretamente ela irá continuar sendo executada indefinidamente. Em alguns casos não é possível encontrar a TF analiticamente, e teremos que calculá-la numericamente usando o [SciPy](https://scipy.org).

In [ ]:
# importando método para calcular integral definida
from scipy.integrate import quad

Vamos definir a função à qual queremos aplicar a transformada de Fourier e uma função para calcular a transformada de Fourier

In [ ]:
def x(t, c):
    return np.exp(-c*t**2)*np.sin(c*t)*t**4

def calc_TF_x(x, f, c):
    # scipy não trabalha com numeros complexos ao calcular integrais
    # então, vamos separar as partes real e imaginária do integrando
    x_TF_integrand_real = lambda t: np.real(x(t, c)*np.exp(-2*np.pi*1j*f*t))
    x_TF_integrand_complex = lambda t: np.imag(x(t, c)*np.exp(-2*np.pi*1j*f*t))
    # agora vamos integrar as partes separadamente
    TF_x_real = quad(x_TF_integrand_real, -np.inf, np.inf)[0]
    TF_x_complex = quad(x_TF_integrand_complex, -np.inf, np.inf)[0]
    return TF_x_real + 1j*TF_x_complex

Agora, vamos criar um vetor de frequências e computar as respectivas transformadas.

In [ ]:
f = np.linspace(0, 5, 150)
TF_x = np.vectorize(calc_TF_x)(x, f, c=2)

In [ ]:
plt.plot(f, np.abs(TF_x))
plt.ylabel('$|\hat{x}(f)|$')
plt.xlabel("f")

# 3. Transformada de Fourier discreta (tempo e frequência discretos)

Muito provavelmente você não trabalhará com dados contínuos dentro da Geofísica, mas sim com uma série temporal $x_{t}$ medida durante um intervalo finito de tempo ($N$ medidas ao longo de um tempo $T$ implica que $N\Delta t = T$). Para tanto, irá precisar de uma versão **discreta** da transformada de Fourier. Esse tema será abordado com detalhe nas próximas aulas, e iremos apenas introduzí-lo brevemente aqui. A transformada de Fourier é definida como:

$\large \displaystyle{\hat{x}(f_{n}) = \sum\limits_{k=0}^{N-1}x_t e^{-2\pi if_n(k\Delta t)}\Delta t} = \sum\limits_{k=0}^{N-1}x_t e^{-2\pi ikn/N}\Delta t$

$\large \displaystyle{f_n = \frac{n}{N\Delta t}}$

onde $f_n$ são as chamadas frequências de Fourier, N é o número de medidas e k é a amostra na qual estamos. Vamos criar um sinal e calcular sua transformada de Fourier com o algoritmo FFT.

In [ ]:
T = 1 # segundos
N = 100 # número de medidas
t = np.linspace(0, T, N) # vetor de tempos
dt = t[1] - t[0] 

In [ ]:
freq = 1.
x = 3*np.sin(2*np.pi*freq*t)

freq = 4
x += np.sin(2*np.pi*freq*t)

freq = 7   
x += 0.5* np.sin(2*np.pi*freq*t)

plt.figure(figsize = (8, 6))
plt.plot(t, x)
plt.ylabel('$Amplitude$')
plt.xlabel('$t (s)$')

plt.show()

In [ ]:
# criando um vetor de frequências
f = fftfreq(len(t), dt)
# calculando a FFT
x_FFT = fft(x)

In [ ]:
# plotando
plt.plot(f, np.abs(x_FFT))
plt.ylabel("$|\hat{x}(f)|$")
plt.xlabel("$f (Hz)$")
plt.show()

In [ ]:
# a transformada é simétrica em torno do zero
# vamos plotar apenas metade
plt.plot(f[:N//2], np.abs(x_FFT[:N//2]))
plt.ylabel("$|\hat{x}(f)|$")
plt.xlabel("$f (Hz)$")
plt.show()

In [ ]:
# por fim, vamos dar um zoom nas frequências < 10 Hz

plt.plot(f[:N//2], np.abs(x_FFT[:N//2]))
plt.xlim([0, 10])
plt.ylabel("$|\hat{x}(f)|$")
plt.xlabel("$f (Hz)$")
plt.show()

Percebemos a presença de 3 picos, justamente em torno das frequências que definimos: 1, 3 e 7 Hz. Além disso, a amplitude dos picos reflete as amplitudes que definimos: 3, 1 e 0.5.

**Exercício.** Contamine o sinal anterior com ruído aleatório e avalie como a transformada de Fourier é afetada, plotando o resultado.